Target
- Fast quantum evaluation for parameterized circuit
- 

Technique:
- Parameterized matrix form?
- Gate fusion (cache + step-forward predict)
- Present qubits as merkle-tree

In [1]:
from qiskit import QuantumCircuit

qc = QuantumCircuit(2, name='bell')
qc.h(0)
qc.cx(0, 1)
custom_gate = qc.to_instruction()

new_circ = QuantumCircuit(2)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import math
import qiskit
import qiskit.quantum_info as qi
%load_ext autoreload
%autoreload 2
qc = qiskit.QuantumCircuit(2)
qc.h(0)
qc.cx(0, 1)
qc.ry(0.5, 1)
qc.append(custom_gate, [0, 1])
print(qc.qasm())

OPENQASM 2.0;
include "qelib1.inc";
gate bell q0,q1 { h q0; cx q0,q1; }
qreg q[2];
h q[0];
cx q[0],q[1];
ry(0.5) q[1];
bell q[0],q[1];



In [3]:
class Gate:
    def __init__(self, matrix, name) -> None:
        self.matrix = matrix
        self.name = name
    def get_num_qbits(self):
        return int(math.log2(len(self.matrix)))
    def __mul__(self, gate):
        return Gate(np.kron(self.matrix, gate.matrix), self.name + gate.name)
        

class Qubit:
    def __init__(self, state = [1, 0]) -> None:
        self.state = state
        self.gates = []
    def evol(self):
        self.state = self.gates.pop[0].matrix @ self.state
    def evol_all(self): 
        while self.gates != []:
            self.evol()
class Qutem:
    def __init__(self, qubits) -> None:
        self.qubits = qubits
        self.state = np.zeros(2**len(qubits))
        self.state[0] = 1
        self.size = 2
    def fuse(self):
        for i in range(self.size - 1):
            self.state = np.kron(self.qubits[i].state, self.qubits[i + 1].state)

In [4]:
import sympy as sp

# Define the symbolic variable
theta = sp.symbols('theta')

# Define the X matrix
X = sp.Matrix([
    [0, 1], 
    [1, 0]
])

# Define the Rx matrix with symbolic expressions
Rx = sp.Matrix([
    [sp.cos(theta/2), -sp.I*sp.sin(theta/2)],
    [-sp.I*sp.sin(theta/2), sp.cos(theta/2)]
])

# Compute the product Rx @ X symbolically
Rx_X = (Rx * X * Rx * X)**4

# Display the result in symbolic form
Rx_X


Matrix([
[ sin(theta/2)**8 - 28*sin(theta/2)**6*cos(theta/2)**2 + 70*sin(theta/2)**4*cos(theta/2)**4 - 28*sin(theta/2)**2*cos(theta/2)**6 + cos(theta/2)**8, 8*I*sin(theta/2)**7*cos(theta/2) - 56*I*sin(theta/2)**5*cos(theta/2)**3 + 56*I*sin(theta/2)**3*cos(theta/2)**5 - 8*I*sin(theta/2)*cos(theta/2)**7],
[8*I*sin(theta/2)**7*cos(theta/2) - 56*I*sin(theta/2)**5*cos(theta/2)**3 + 56*I*sin(theta/2)**3*cos(theta/2)**5 - 8*I*sin(theta/2)*cos(theta/2)**7,  sin(theta/2)**8 - 28*sin(theta/2)**6*cos(theta/2)**2 + 70*sin(theta/2)**4*cos(theta/2)**4 - 28*sin(theta/2)**2*cos(theta/2)**6 + cos(theta/2)**8]])

In [16]:
import gate

for thetas in np.linspace(0, 0.5, 10000):
    a = (gate.gate['Rx'](thetas) @ gate.gate['X'])
    #print(a)

In [17]:
import sympy
a = (Rx @ X)
f = sympy.lambdify(theta, a, "numpy")
print(a)

Matrix([[-I*sin(theta/2), cos(theta/2)], [cos(theta/2), -I*sin(theta/2)]])


In [18]:
for thetas in np.linspace(0, 0.5, 100000):
    # Rx_X_evaluated = Rx_X.subs(theta, thetas).evalf()
    f(thetas)